# Тетрадка для проведения кластеризации

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pandas as pd 
import numpy as np 

import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

from matplotlib import rcParams
plt.style.use('default')
plt.rcParams['axes.prop_cycle'] = plt.rcParamsDefault['axes.prop_cycle']

pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 150)

from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.cluster import KMeans, DBSCAN, SpectralClustering, MeanShift
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from sklearn import metrics 
from sklearn.metrics import pairwise_distances
from scipy.spatial.distance import cdist

import warnings
warnings.filterwarnings("ignore")

In [25]:
x_2023 = pd.read_excel('full_dfs/x_2023.xlsx')

In [6]:
# Заполняем пропуски при помощи KNN:
# from clustering_functions import knn_nan_inputer
# x_2023_knn = knn_nan_inputer(x_2023)

In [ ]:
# silhouette_score = metrics.silhouette_score(self.features_table, labels, metric='euclidean')

In [ ]:
#скоры

In [29]:
x_2023.describe()

,Н1 CAR,Н2 liquidity,Н3 liquidity,ROA,ROE,NPL Ratio,Debt/TotalAssets,Deposits/TotalAssets,TotalLoans/TotalAssets,LDR,LiquidAssetsRatio,Z-score,КредитыЮЛ/TotalAssets,КредитыФЛ/TotalAssets,ВыданныеМБК/TotalAssets,ПривлеченныеМБК/TotalAssets,Капитал/Активы,log_TotalAssets,gos_sobstv,foreign,system
count,336.000000,230.000000,328.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,333.000000,339.000000,326.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000,339.000000
mean,0.505829,1.374959,1.610101,0.026296,-0.364052,0.008651,0.321466,0.476693,0.667327,0.006176,0.137834,0.025814,0.214669,0.106797,0.345861,0.066227,0.240910,16.565817,0.053097,0.094395,0.032448
std,0.670366,1.300844,1.134924,0.056425,8.835744,0.069839,0.238185,0.230237,0.227530,0.063839,0.170910,0.059523,0.186865,0.153801,0.258393,0.313916,0.353005,2.265031,0.224559,0.292810,0.177450
min,0.000000,0.000000,0.000000,-0.248132,-162.453643,0.000000,0.000000,0.000000,0.000000,0.000000,0.001112,0.000000,0.000000,0.000000,0.000000,0.000000,-5.217186,11.597055,0.000000,0.000000,0.000000
25%,0.168500,0.667900,1.011600,0.008699,0.040116,0.000000,0.095208,0.337486,0.544326,0.000219,0.048470,0.007463,0.033504,0.002700,0.139650,0.000000,0.128559,14.817675,0.000000,0.000000,0.000000
50%,0.299250,0.975850,1.306350,0.018523,0.092746,0.000104,0.331125,0.528217,0.730677,0.000277,0.078463,0.014384,0.183898,0.037977,0.293845,0.000000,0.196721,16.389328,0.000000,0.000000,0.000000
75%,0.534800,1.634350,1.905350,0.034076,0.168815,0.000816,0.518282,0.646407,0.832468,0.000419,0.138108,0.028987,0.369075,0.149529,0.532911,0.027758,0.349801,17.998877,0.000000,0.000000,0.000000
max,5.683800,8.724800,9.709500,0.475582,2.384226,1.000000,0.929828,0.880495,0.988576,1.000000,0.975872,1.000000,0.747158,0.929790,0.981356,5.277712,0.975964,24.649107,1.000000,1.000000,1.000000


In [30]:
from clustering_functions import K_means_cluster

In [31]:
X = x_2023.drop(columns = ['bank_name', 'log_TotalAssets'])
bank_names =x_2023['bank_name'] 

In [ ]:
# силуэт [-1, 1] - чем ближе к 1, тем лучше разбиение

In [32]:
k_means = K_means_cluster(features_table = X, 
                          num_clusters = [4], 
                          bank_name_column = bank_names, 
                          scaling = False,
                         pca_or_tsne='tsne', #dimensional reduce with tsne or pca
                             clustering_before_dimreduce = True,
                          perplexity=20 #for tsne
                         )
k_means.k_means_clustering()

silhouette_score: 0.3987052263311908
calinski_harabasz_score: 4126.431845014191
davies_bouldin_score: 0.7925865782158029


In [45]:
names_list, dataframe = k_means.compute_cluster_stats(list_index = 0, cluster_index = 3)

Всего кластеров 4
Банков попало в кластер: 6


In [46]:
names_list[:20]

array(['БМ-Банк', 'Ренессанс Банк', 'Контур.Банк', '«ГОРБАНК»',
       'СОЦИУМ-БАНК', 'Креди Агриколь КИБ'], dtype=object)

In [47]:
dataframe

,mean,std,min,max,25%,50%,75%
Н1 CAR,0.812433,0.715551,0.109200,2.199700,0.559175,0.633550,0.731550
Н2 liquidity,7.074500,1.061645,6.113500,8.724800,6.262300,6.824400,7.447500
Н3 liquidity,5.822067,2.477170,2.852400,9.709500,4.120775,5.794500,6.862600
ROA,0.021532,0.024253,-0.012462,0.061800,0.013670,0.018884,0.027336
ROE,0.042447,0.074601,-0.102562,0.098181,0.040686,0.067288,0.086325
NPL Ratio,0.101083,0.240347,0.000000,0.591575,0.000038,0.000799,0.010358
Debt/TotalAssets,0.163744,0.273627,0.003940,0.715356,0.018107,0.077381,0.101888
Deposits/TotalAssets,0.482403,0.225384,0.065214,0.752323,0.496357,0.520138,0.541166
TotalLoans/TotalAssets,0.680978,0.160587,0.369205,0.814908,0.683017,0.726641,0.766660
LDR,0.000438,0.000377,0.000218,0.001203,0.000280,0.000288,0.000339
